In [2]:
import tensorflow as tf
# import tensorflow_decision_forests as tfdf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pl
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df.drop(columns=["Id"], inplace=True)
# df.isna().sum().sort_values(ascending=False)[:20]
df_test.drop(columns=["Id"], inplace=True)

In [5]:
df.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], inplace=True)
df_test.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], inplace=True)

In [6]:
for i in df_test.columns:
    if df_test[i].isna().any():
        if df_test[i].dtypes == 'object':
            df_test[i].fillna(df_test[i].mode()[0], inplace=True)
        else:
            df_test[i].fillna(df_test[i].median(), inplace=True)

for i in df.columns:
    if df[i].isna().any():
        if df[i].dtypes == 'object':
            df[i].fillna(df[i].mode()[0], inplace=True)
        else:
            df[i].fillna(df[i].median(), inplace=True)
            
            

In [7]:
len(df_test)

1459

In [8]:
combined = pd.concat([df, df_test], axis=0)

In [9]:
combined

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,6,2006,WD,Normal,NaN
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,4,2006,WD,Abnorml,NaN
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2006,WD,Abnorml,NaN
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,700,7,2006,WD,Normal,NaN


In [10]:
df["LotArea"][1459]

9937

In [11]:
combined = pd.get_dummies(combined, drop_first=True)

In [12]:
train_encoded = combined.iloc[:len(df), :]
test_encoded = combined.iloc[len(df):, :]

In [13]:
test_encoded.isna().sum().sort_values(ascending=True)

MSSubClass                  0
MasVnrType_None             0
MasVnrType_Stone            0
ExterQual_Fa                0
ExterQual_Gd                0
                         ... 
Condition1_RRAe             0
Condition1_RRAn             0
Neighborhood_Sawyer         0
SaleCondition_Partial       0
SalePrice                1459
Length: 233, dtype: int64

In [14]:
X_cleaned = train_encoded.drop("SalePrice", axis=1)
y_cleaned = train_encoded["SalePrice"]

X_tested = test_encoded.drop("SalePrice", axis=1)

In [15]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_cleaned, y_cleaned, test_size=.25, random_state=1111)

In [16]:
X_train_clean.shape

(1095, 232)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Normalization
from tensorflow.keras.optimizers import Adam


normalizer = Normalization(axis=-1)
normalizer.adapt(np.array(X_train_clean))

model = Sequential()
# model.add(normalizer)
model.add(Dense(64, activation='relu', input_shape=(232, )))
model.add(Dense(64, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.1), loss='mse', metrics=["mean_absolute_error"])

In [38]:
history = model.fit(X_train_clean, y_train_clean, batch_size=128, epochs=100, validation_data=(X_test_clean, y_test_clean))

Epoch 1/100
9/9 [==============================] - 1s 22ms/step - loss: 24841201664.0000 - mean_absolute_error: 116668.0391 - val_loss: 22853244928.0000 - val_mean_absolute_error: 133212.6094
Epoch 2/100
9/9 [==============================] - 0s 7ms/step - loss: 11700226048.0000 - mean_absolute_error: 87661.8359 - val_loss: 3342805760.0000 - val_mean_absolute_error: 43504.7617
Epoch 3/100
9/9 [==============================] - 0s 7ms/step - loss: 5644869120.0000 - mean_absolute_error: 52009.6953 - val_loss: 5148318720.0000 - val_mean_absolute_error: 64207.0469
Epoch 4/100
9/9 [==============================] - 0s 7ms/step - loss: 4762185216.0000 - mean_absolute_error: 50895.6836 - val_loss: 2935383040.0000 - val_mean_absolute_error: 36942.6016
Epoch 5/100
9/9 [==============================] - 0s 7ms/step - loss: 3362038528.0000 - mean_absolute_error: 42500.9375 - val_loss: 3450779648.0000 - val_mean_absolute_error: 37901.6602
Epoch 6/100
9/9 [==============================] - 0s 6ms/s

In [39]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 64)                14912     
                                                                 
 dense_28 (Dense)            (None, 64)                4160      
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                                 
Total params: 19,137
Trainable params: 19,137
Non-trainable params: 0
_________________________________________________________________


In [40]:
result = model.predict(X_tested)

46/46 [==============================] - 0s 1ms/step


In [41]:
result

array([[137894.12],
       [127106.21],
       [194503.75],
       ...,
       [188502.03],
       [129292.97],
       [223877.8 ]], dtype=float32)

In [42]:
df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,32,0,0,0,0,700,7,2006,WD,Normal


In [43]:
test_encoded

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,1,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,0,1,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1455,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1456,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1457,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,...,0,0,0,0,1,0,0,0,1,0


In [45]:
temp_test = pd.read_csv('test.csv')
temp_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [73]:

final = pd.DataFrame(temp_test["Id"])
final["SalePrice"] = result.flatten().tolist()
final

,Id,SalePrice
0,1461,137894.125000
1,1462,127106.210938
2,1463,194503.750000
3,1464,203201.421875
4,1465,187259.203125
...,...,...
1454,2915,89043.070312
1455,2916,99668.773438
1456,2917,188502.031250
1457,2918,129292.968750


In [74]:
final.to_csv("result_tf.csv", index=False)